In [ ]:
!pip install torcheval
!pip install tabulate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.4/158.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 11.2 MB/s eta 0:00:00


In [ ]:

from google.colab import drive
import torch
from torch.utils.data import Dataset, DataLoader, default_collate
import h5py
import numpy as np
import librosa as lib
import librosa.display as libd
import pandas as pd
from tabulate import tabulate
import copy

drive.mount('/content/drive')
plots_dir = '/content/drive/My Drive/Plots/full'
models_dir = '/content/drive/My Drive/Models/full'
maps_dir = '/content/drive/My Drive/Map'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class STEADDataset(Dataset):

  def __init__(self,csv_file,hdf5_file,transform=None):
    """
      Args:
        csv_file (string): calea catre fisierul csv cu etichete.
        hdf5_file (string): fisierul cu formele de unda.
        transform (callable, optional): transformare a unui esantion.
    """
    self.tags = pd.read_csv(csv_file)
    self.hdf5_file = hdf5_file
    self.traces = self.tags['trace_name'].to_list()
    self.transform = transform

  def __len__(self):
    return len(self.traces)

  def __getitem__(self,idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    dataset = h5py.File(self.hdf5_file,'r')
    tracename = self.traces[idx]
    waveform = dataset.get('data/'+tracename)
    data = np.array(waveform)
    spectrograms = self.getSpectrogram(data)

    sample = {'spectrograms':spectrograms,
              'source_magnitude':waveform.attrs['source_magnitude'],
              'source_latitude':waveform.attrs['source_latitude'],
              'source_longitude':waveform.attrs['source_longitude'],
              'source_depth_km':waveform.attrs['source_depth_km']}

    if self.transform:
      sample = self.transform(sample)

    dataset.close()
    return sample

  def getSpectrogram(self, waveforms):
    # defining axis
    EW = waveforms[:,0]
    NS = waveforms[:,1]
    Vert = waveforms[:,2]

    EW_ft = lib.stft(EW,n_fft=1024, hop_length=32)
    NS_ft = lib.stft(NS,n_fft=1024, hop_length=32)
    Vert_ft = lib.stft(Vert,n_fft=1024, hop_length=32)

    EW_db = lib.amplitude_to_db(np.abs(EW_ft), ref=np.max)
    NS_db = lib.amplitude_to_db(np.abs(NS_ft), ref=np.max)
    Vert_db = lib.amplitude_to_db(np.abs(Vert_ft), ref=np.max)

    spectrograms = np.array([EW_db, NS_db, Vert_db])

    return spectrograms

class ToTensor(object):
    """Converteste NumPY ndarrays la Tensori."""

    def __call__(self, sample):
      if sample['source_depth_km'] == "None":
        return None
      else:
        spectrograms = sample['spectrograms']
        results = np.array([sample['source_magnitude'],
                            sample['source_latitude'],
                            sample['source_longitude'],
                            sample['source_depth_km']], dtype=np.float32)

        return {'spectrograms': torch.from_numpy(spectrograms),
                'results':  torch.from_numpy(results)}


In [ ]:
def custom_collate_fn(batch):
  # filtrez estanioanele None (regasite in cazul etichetei adancimii)
  filtered_batch = [sample for sample in batch if sample is not None]
  if len(filtered_batch) == 0:
    # daca lotul nu are niciun esantion – toate None
    return None
  else:
    # creez noul batch cu esantioanele None eliminate
    return default_collate(filtered_batch)

In [ ]:
STEAD_dataset = STEADDataset(csv_file="drive/My Drive/dataset.csv",
                             hdf5_file="drive/My Drive/dataset.hdf5",
                             transform=ToTensor())

<ipython-input-36-c971ea9a199e>:10: DtypeWarning:

Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
from torch.nn import Module # Implementez o clasa in locul unui model secvential
from torch.nn import Conv2d # strat convolutional
from torch.nn import BatchNorm2d # Normalizarea loturilor
from torch.nn import Linear # functia de regresie liniara
from torch.nn import MaxPool2d # esantionare 2D - MaxPooling
from torch.nn import ReLU # functie de activare
from torch.optim import Adam # Optimizator Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau # programator al ratei de invatare
from torch import flatten # redimensionarea in start fc 1D
from torch.utils.data import random_split # pentru impartirea dataset-ului
from torcheval.metrics import R2Score # evalueaza precizia modelului
from torch import nn
import torch
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt


In [ ]:
# definesc blocul rezidual
class block(Module):
  def __init__(self, in_channels, out_channels, kernel_size=(5,5),
               padding=(2,2)):
    super(block, self).__init__()
    self.conv1 = Conv2d(in_channels, out_channels,
                        kernel_size = kernel_size, padding=padding)
    self.bn1 = BatchNorm2d(out_channels)
    self.conv2 = Conv2d(out_channels, out_channels,
                        kernel_size= kernel_size, padding=padding)
    self.bn2 = BatchNorm2d(out_channels)
    self.relu = ReLU(inplace=True)
    self.maxpool = MaxPool2d(kernel_size=(2,2), stride=(2,2))

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    identity = x
    x = self.conv2(x)
    x = self.bn2(x)
    x += identity

    x = self.relu(x)

    output = self.maxpool(x)

    return output

class ResNet(Module):
  def __init__(self, block, numChannels, outputNodes):
    super(ResNet, self).__init__()
    # initializez primul strat CONV => RELU => POOL
    self.conv1 = Conv2d(in_channels=numChannels, out_channels=16,
                        kernel_size=(7,7), padding=(3,3))
    self.bn1 = BatchNorm2d(16)
    self.relu = ReLU(inplace=True)
    self.maxpool = MaxPool2d(kernel_size=(2,2), stride=(2,2))

    # initializez al doilea strat CONV => RELU => POOL
    self.conv2 = Conv2d(in_channels=16, out_channels=16, kernel_size=(7,7),
                        padding=(3,3))
    self.bn2 = BatchNorm2d(16)

    # straturile de tip ResNet
    self.layer1 = block(in_channels=16, out_channels=32, kernel_size=(5,5),
                        padding=(2,2))
    self.layer2 = block(in_channels=32, out_channels=64, kernel_size=(3,3),
                        padding=(1,1))
    self.layer3 = block(in_channels=64, out_channels=96, kernel_size=(3,3),
                        padding=(1,1))

    # initializez ultimul strat CONV => RELU => CONV => POOL
    self.conv3 = Conv2d(in_channels=96, out_channels=128, kernel_size=(3,3),
                        padding=(1,1))
    self.bn3 = BatchNorm2d(128)

    self.conv4 = Conv2d(in_channels=128, out_channels=128,
                        kernel_size=(3,3), padding=(1,1))
    self.bn4 = BatchNorm2d(128)

    # initializez stratul FC => ReLU layers - fully connected
    self.fc1 = Linear(in_features=2048, out_features=1024)

    # initializez stratul FC => Linear *Regression*
    self.fc2 = Linear(in_features=1024, out_features=outputNodes)

  def forward(self, x):
  # trimit intrarea prin primul bloc CONV -> ReLU -> POOL
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    # trimit rezultatele prin al doilea bloc CONV -> ReLU -> POOL
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.relu(x)
    x = self.maxpool(x)

    # trimit datele prin blocurile reziduale
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)

    # trimit datele prin blocul CONV => RELU => CONV => POOL
    x = self.conv3(x)
    x = self.bn3(x)
    x = self.relu(x)
    x = self.conv4(x)
    x = self.bn4(x)
    x = self.maxpool(x)

    # redimensionez iesirea stratului anterior si trimit catre stratul ascuns FC
    x = flatten(x, 1)
    x = self.fc1(x)
    x = self.relu(x)

    # trimit datele catre stratul Linear pentru realizarea regresiei
    output = self.fc2(x)

    # intorc rezultatele prezise
    return output



# definesc clasa EarlyStopping
class EarlyStopping():
  def __init__(self, patience = 1, min_delta = 0):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation_loss = np.inf
    self.best_epoch = 0
    self.best_train = [None] * 5
    self.best_val = [None] * 5

  def earlyStop(self, validation_loss, epoch, TrainLoss, ValLoss):
    if validation_loss <= self.min_validation_loss:
      print("[INFO] In EPOCH {} the loss value improved from {:.5f} to {:.5f}".format(epoch, self.min_validation_loss, validation_loss))
      self.min_validation_loss = validation_loss
      self.counter = 0
      self.best_epoch = epoch
# salvez ponderile celui mai performant model de pana acum
      torch.save(model.state_dict(), f"{models_dir}/ResNet_state_dict.pt")
      self.setBestLosses(TrainLoss, ValLoss)

    elif validation_loss > (self.min_validation_loss + self.min_delta):
      self.counter += 1
      print("[INFO] In EPOCH {} the loss value did not improve from {:.5f}. This is the {} EPOCH in a row.".format(epoch, self.min_validation_loss,
                                  self.counter))
      if self.counter >= self.patience:
        return True
    return False

  def setCounter(self, counter_state):
    self.counter = counter_state

  def setMinValLoss(self, ValLoss):
    self.min_validation_loss = ValLoss

  def setBestLosses(self, TrainLoss, ValLoss):
    self.best_train = TrainLoss
    self.best_val = ValLoss

  def setBestEpoch(self, bestEpoch):
    self.best_epoch = bestEpoch

  def getBestTrainLosses(self):
    return self.best_train

  def getBestValLosses(self):
    return self.best_val

  def getBestEpoch(self):
    return self.best_epoch

  def saveLossesLocally(self):
    np.save(f'{models_dir}/losses_train.npy', np.array(self.best_train))
    np.save(f'{models_dir}/losses_val.npy', np.array(self.best_val))

  def loadLossesLocally(self):
    self.best_train = np.load(f'{models_dir}/losses_train.npy')
    self.best_val = np.load(f'{models_dir}/losses_val.npy')


In [ ]:
# definesc hiperparametrii
INIT_LR = 1*1e-4
BATCH_SIZE = 5
EPOCHS = 50

# definesc impartirea set de antrenare-validare-testare
VAL_TEST_SPLIT = 0.06/100
TRAIN_SPLIT = 1 - VAL_TEST_SPLIT

# aleg dispozitivul pe care vor avea loc operatiile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] device used for training...{}".format(device))

[INFO] device used for training...cuda


In [ ]:
# calculez numarul de esantioane pentru fiecare set de date
print("[INFO] generating the train/validation split...")
numTrainSamples = int(len(STEAD_dataset)*TRAIN_SPLIT)
# numValSamples = int(len(STEAD_dataset)*VAL_TEST_SPLIT)
numTestSamples = int(len(STEAD_dataset)-(numTrainSamples))
# fac impartirea aleatoare a esantioanelor
(trainData, testData) = random_split(STEAD_dataset,[numTrainSamples, numTestSamples], generator=torch.Generator().manual_seed(19))

[INFO] generating the train/validation split...


In [ ]:
# initializez setul de antrenare, validare (comentat), testare
trainDataLoader = DataLoader(trainData, shuffle=True, batch_size=BATCH_SIZE, collate_fn = custom_collate_fn)
# valDataLoader = DataLoader(valData, batch_size=BATCH_SIZE, collate_fn = custom_collate_fn)
testDataLoader = DataLoader(testData, batch_size=BATCH_SIZE, collate_fn = custom_collate_fn)

# calculez numarul de pasi in functie de dimensiunea lotului de date de intrare
trainSteps = len(trainDataLoader.dataset) // BATCH_SIZE
# valSteps = len(valDataLoader.dataset) // BATCH_SIZE
testSteps = len(testDataLoader.dataset) // BATCH_SIZE

In [ ]:
# initializez modelul ResNet-10
print("[INFO] initializing the ResNet model...")
model = ResNet(block=block, numChannels=3, outputNodes=4).to(device)

# initializez functia de pierdere si coeficientul de determinare R2
lossFn = nn.MSELoss()
r2score_metric = R2Score(multioutput='raw_values', device=device)

# incarc ponderile si incep evaluarea retelei neurale
print("[INFO] evaluating network...")
model.load_state_dict(torch.load(f"{models_dir}/ResNet_state_dict.pt"))

[INFO] initializing the ResNet model...
[INFO] evaluating network...


<All keys matched successfully>

In [ ]:
test_results = {
    "true_values":[],
    "pred_values":[]
}

# opresc autograd pentru testarea retelei
with torch.no_grad():
  # setez modeleul in modul evaluare
  model.eval()

  # initializez costurile totale pe setul de test
  generalTestLoss = 0
  magnitudeTestLoss = 0
  latitudeTestLoss =  0
  longitudeTestLoss = 0
  depthTestLoss = 0

  # interez prin setul de test
  for sampled_batch in testDataLoader:
    if sampled_batch is None:
      continue

    # trimit datele de intrare catre dispozitiv
    (x, y) = (sampled_batch['spectrograms'].to(device),sampled_batch['results'].to(device))
    test_results["true_values"].append(y.cpu().detach().numpy().tolist())

    # fac estimarile si le adaug in lista
    pred = model(x)
    test_results["pred_values"].append(pred.cpu().detach().numpy().tolist())

    generalTestLoss += lossFn(pred, y).cpu().detach().numpy()
    magnitudeTestLoss += lossFn(pred[:,0], y[:,0]).cpu().detach().numpy()
    latitudeTestLoss += lossFn(pred[:,1], y[:,1]).cpu().detach().numpy()
    longitudeTestLoss += lossFn(pred[:,2], y[:,2]).cpu().detach().numpy()
    depthTestLoss += lossFn(pred[:,3], y[:,3]).cpu().detach().numpy()
    r2score_metric.update(pred, y)

  # generez MSE si scorul R2 pe setul de test
  avgGeneralTestLoss = generalTestLoss / testSteps
  avgMagnitudeTestLoss = magnitudeTestLoss / testSteps
  avgLatitudeTestLoss =  latitudeTestLoss / testSteps
  avgLongitudeTestLoss = longitudeTestLoss / testSteps
  avgDepthTestLoss =  depthTestLoss / testSteps

  r2score_value = r2score_metric.compute()

  print("[INFO] Loss/Accuracy values obtained on the test set")
  print("[INFO] Test loss (General, Magnitude, Latitude, Longitude, Depth): {:.5f}, {:.5f} , {:.5f} , {:.5f}, {:.5f}".format(
      avgGeneralTestLoss, avgMagnitudeTestLoss, avgLatitudeTestLoss, avgLongitudeTestLoss, avgDepthTestLoss))
  print("[INFO] R2 Score obtained on the test set: {}".format(r2score_value.cpu().detach().numpy()))

[INFO] Loss/Accuracy values obtained on the test set
[INFO] Test loss (General, Magnitude, Latitude, Longitude, Depth): 0.81112, 0.05462 , 0.02166 , 0.03019, 3.13801
[INFO] R2 Score obtained on the test set: [0.68294966 0.7292813  0.57623273 0.78304136]


In [ ]:
# realizez și salvez graficele de dispersie pentru Y_observat - Y_estimat
plt.style.use("ggplot")

test_true = []
test_pred = []

for i in range(len(test_results["true_values"])):
  test_true.extend(test_results["true_values"][i])
  test_pred.extend(test_results["pred_values"][i])

test_true = np.array(test_true)
test_pred = np.array(test_pred)

# realizez graficul dispersiei punctelor de magnitudine Y_observat - Y_estimat
plt.figure("magnitude_true-pred").clear()
plt.plot(test_true[:,0], test_pred[:,0], "ob")
m, b = np.polyfit(test_true[:,0], test_pred[:,0], 1)
plt.plot(test_true[:,0], m*test_true[:,0]+b,"--r")
plt.title("Magnitudine Y_estimat vs Y_observat")
plt.xlabel("Y_observat")
plt.ylabel("Y_estimat")
plt.savefig(f"{plots_dir}/ResNet3_magnitudine_obs-est.png")

# realizez graficul dispersiei punctelor de latitudine Y_observat - Y_estimat
plt.figure("latitude_true-pred").clear()
plt.plot(test_true[:,1], test_pred[:,1], "ob")
m, b = np.polyfit(test_true[:,1], test_pred[:,1], 1)
plt.plot(test_true[:,1], m*test_true[:,1]+b,"--r")
plt.title("Latitudine Y_estimat vs Y_observat")
plt.xlabel("Y_observat")
plt.ylabel("Y_estimat")
plt.savefig(f"{plots_dir}/ResNet3_latitudine_obs-est.png")

# realizez graficul dispersiei punctelor de longitudine Y_observat - Y_estimat
plt.figure("longitude_true-pred").clear()
plt.plot(test_true[:,2], test_pred[:,2], "ob")
m, b = np.polyfit(test_true[:,2], test_pred[:,2], 1)
plt.plot(test_true[:,2], m*test_true[:,2]+b,"--r")
plt.title("Longitudine Y_estimat vs Y_observat")
plt.xlabel("Y_observat")
plt.ylabel("Y_estimat")
plt.savefig(f"{plots_dir}/ResNet3_longitudine_obs-est.png")

# realizez graficul dispersiei punctelor de adancime Y_observat - Y_estimat
plt.figure("depth_true-pred").clear()
plt.plot(test_true[:,3], test_pred[:,3], "ob")
m, b = np.polyfit(test_true[:,3], test_pred[:,3], 1)
plt.plot(test_true[:,3], m*test_true[:,3]+b,"--r")
plt.title("Adancime Y_estimat vs Y_observat")
plt.xlabel("Y_observat")
plt.ylabel("Y_estimat")
plt.savefig(f"{plots_dir}/ResNet3_adancime_obs-est.png")


# generating table with best values obtained on train, evaluation and test
early_stopper = EarlyStopping(patience = 4)
early_stopper.loadLossesLocally()

models_performance = [] # used to showcase the Loss/Accuracy values obtained
models_performance.append(["MSE General", round(early_stopper.getBestTrainLosses()[0], 5),  round(early_stopper.getBestValLosses()[0], 5), avgGeneralTestLoss])
models_performance.append(["MSE Magnitudine", round(early_stopper.getBestTrainLosses()[1], 5),  round(early_stopper.getBestValLosses()[1], 5), avgMagnitudeTestLoss])
models_performance.append(["MSE Latitudine", round(early_stopper.getBestTrainLosses()[2], 5),  round(early_stopper.getBestValLosses()[2], 5), avgLatitudeTestLoss])
models_performance.append(["MSE Longitudine", round(early_stopper.getBestTrainLosses()[3], 5),  round(early_stopper.getBestValLosses()[3], 5), avgLongitudeTestLoss])
models_performance.append(["MSE Adâncime", round(early_stopper.getBestTrainLosses()[4], 5),  round(early_stopper.getBestValLosses()[4], 5), avgDepthTestLoss])

r2_score_list = r2score_value.cpu().detach().numpy().tolist()
models_performance.append(["R2 Score", r2_score_list[0], r2_score_list[1], r2_score_list[2], r2_score_list[3]])

print("----------------------------------------------------------------------")
print("Costurile obtinute pe loturile de antrenare, validare si testare")
print(tabulate(models_performance[0:5], headers=["Metrică", "Antrenare", "Validare", "Testare"], tablefmt="github"))

print("----------------------------------------------------------------------")
print("Precizia modelului pe setul de test")
print(tabulate([models_performance[5]], headers=["Metrică", "Magnitudine", "Latitudine", "Longitudine", "Adâncime"], tablefmt="github"))

# serialize the model to disk
# torch.save(model, f"{models_dir}/ResNet_model.pt")

----------------------------------------------------------------------
Costurile obtinute pe loturile de antrenare, validare si testare
| Metrică         |   Antrenare |   Validare |   Testare |
|-----------------|-------------|------------|-----------|
| MSE General     |     0.70137 |    1.11172 | 0.811121  |
| MSE Magnitudine |     0.03283 |    0.03244 | 0.0546233 |
| MSE Latitudine  |     0.03281 |    0.03493 | 0.0216575 |
| MSE Longitudine |     0.04325 |    0.04592 | 0.0301934 |
| MSE Adâncime    |     2.6966  |    4.33358 | 3.13801   |
----------------------------------------------------------------------
Precizia modelului pe setul de test
| Metrică   |   Magnitudine |   Latitudine |   Longitudine |   Adâncime |
|-----------|---------------|--------------|---------------|------------|
| R2 Score  |       0.68295 |     0.729281 |      0.576233 |   0.783041 |


In [ ]:
import plotly.graph_objects as go

# definesc datele pentru valorile reale (gt) si cele estimate (pred)
gt_magnitudes = test_true[:,0]
gt_latitudes = test_true[:,1]
gt_longitudes = test_true[:,2]
gt_depth = test_true[:,3]

pred_magnitudes = test_pred[:,0]
pred_latitudes = test_pred[:,1]
pred_longitudes = test_pred[:,2]
pred_depth = test_pred[:,3]

# definesc aspectul hartii
layout = go.Layout(
    mapbox=dict(
        center=dict(lat=33.5, lon=-116.8),
        zoom=8,
        style='stamen-terrain'
    ),
    title='Epicentrele cutremurelor observate și estimate')

# definesc textul afisat la vizualizare punctelor
gt_hover_text = ['Numar cutremur: {} <br>Latitudine: {}<br>Longitudine: {} <br>Magnitudine: {} <br>Adancime: {}'.format(num, lat, lon, magn, depth)
                 for num, (lat, lon, magn, depth) in enumerate(zip(gt_latitudes, gt_longitudes, gt_magnitudes, gt_depth))]
pred_hover_text = ['Numar cutremur: {} <br>Latitudine: {}<br>Longitudine: {} <br>Magnitudine: {} <br>Adancime: {}'.format(num, lat, lon, magn, depth)
                   for num, (lat, lon, magn, depth) in enumerate(zip(pred_latitudes, pred_longitudes, pred_magnitudes, pred_depth))]

# creez cutiile hartii de dispersie a coord. epicentrelor reale si estim.
gt_trace = go.Scattermapbox(
    lat=gt_latitudes,
    lon=gt_longitudes,
    mode='markers',
    marker=dict(
        color='blue'
    ),
    name='Date seismice observate',
    hovertext=gt_hover_text,
    hoverinfo='text'
)

pred_trace = go.Scattermapbox(
    lat=pred_latitudes,
    lon=pred_longitudes,
    mode='markers',
    marker=dict(
        color='red'
    ),
    name='Date seismice estimate',
    hovertext=pred_hover_text,
    hoverinfo='text'
)

# creez figura, adaug punctele si aspectul
fig = go.Figure(data=[gt_trace, pred_trace], layout=layout)

# salvez harta ca fisier html
fig.write_html(f"{maps_dir}/Harta_ResNet3.html")